# Contrastive Language-Image Pretraining with SogCLR

### **Introduction**

In this tutorial, you will learn how to conduct contrastive language-image pretraining by optimizing the [Global Contrastive Loss](https://arxiv.org/abs/2202.12387) (GCL) on a subset of the [Conceptual Captions](https://ai.google.com/research/ConceptualCaptions/) dataset. Also, you will learn how to evaluate the model on retrieval task using the [MSCOCO](https://cocodataset.org/#home) dataset and zero-shot classification task using the [ImageNet](https://www.image-net.org/challenges/LSVRC/index.php) dataset. The code is based on [iSogCLR's](https://github.com/zhqiu/contrastive-learning-iSogCLR) codebase, which includes the implementation of CLIP, SogCLR and iSogCLR.

### Preparation

First, we:

1. Download the source code and data
2. Install required packages

### Training

The following command runs the training script to train a ResNet50 (pretrained on ImageNet) and a DistilBERT (pretrained on BookCorpus and English Wikipedia) on the cc3m dataset using the SogCLR loss for 30 epochs with temperature 0.01.

In [7]:
!python "D:\projects\improved-clip\src\improved-clip\main.py" \
    --data_path "D:\projects\improved-clip\src\datasets"  \
    --ann_path "D:\projects\improved-clip\src\datasets\clip_train" \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir "D:\projects\improved-clip\src\output\sogclr_cc3m_g0.8_e30" \
    --init_model \
    --use_amp \
    --ita_type sogclr \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --device cpu \
    --epochs 1

Creating retrieval dataset
len of train_dataset: 100000
len of validation dataset: 5000
Creating model


Traceback (most recent call last):
  File "D:\projects\improved-clip\src\improved-clip\main.py", line 214, in <module>
    run_pipeline(args)
  File "D:\projects\improved-clip\src\improved-clip\main.py", line 75, in run_pipeline
    model = CLIP(
  File "D:\projects\improved-clip\src\improved-clip\models\model_clip.py", line 87, in __init__
    self.criterion = SogCLR_Loss(world_size=world_size, gamma=sogclr_gamma, temperature=self.temp, bsz=bsz)
  File "D:\projects\improved-clip\src\improved-clip\models\losses.py", line 72, in __init__
    self.s_I = torch.zeros(N).cuda()
  File "d:\projects\improved-clip\src\.venv\lib\site-packages\torch\cuda\__init__.py", line 247, in _lazy_init
    torch._C._cuda_init()
RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx


### Evaluation

The following command runs the evaluation script to evaluate the retrieval performance of the trained model on the MSCOCO validation dataset and the zero-shot classification performance on the ImageNet validation dataset. The evaluation command is obtained by appending `--evaluate --checkpoint /path/to/your/checkpoint --zs_dataset imagenet --zs_datafolder /path/to/imagenet/val` to the training command.

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python "D:\projects\improved-clip\src\improved-clip\main.py" \
    --data_path "D:\projects\improved-clip\src\datasets" \
    --ann_path "D:\projects\improved-clip\src\datasets\clip_train" \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir output/isogclr_cc3m_g0.8_e30 \
    --init_model \
    --use_amp \
    --ita_type sogclr \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --epochs 1 \
    --evaluate --checkpoint './output/sogclr_cc3m_g0.8_e30/checkpoint_25.pth' \
    --zs_dataset imagenet --zs_datafolder ./datasets/imagenet/val

: 

### Benchmarks

The following results are recall at 1 results on the provided MSCOCO and ImageNet datasets. The first row of results are from the model trained using the CLIP loss, and the second row of results are from the model trained using the SogCLR loss. All results are based on a batch size of 128 for 30-epoch pretraining. IR@1 denotes the recall at 1 of image retrieval on MSCOCO, TR@1 denotes the recall at 1 of text retrieval on MSCOCO, and ACC@1 denotes the top 1 accuracy on ImageNet. Average denotes the average of the three metrics.

| Method | MSCOCO TR@1 | MSCOCO IR@1 | ImageNet ACC@1 | Average |
|:----------:|:--------:|:--------:|:--------:|:--------:|
| CLIP | 12.0 | 9.32 | 21.35 | 14.22 |
| SogCLR |  14.38  |  10.73  | 24.54 | 16.55 |